# Initial Repetition Recommender system
### Data preperation
First I'll load necessary data and find which ccid's were repeated.
I'm ignoring video_id for now since I need more exploration of it (e.g. how it is distributed).
Though using video_id might contextualise better repetition need, depending on how it was used.

Firstly the itemset will only be the videos which have been repeated.
I will then expand the item set with every video ccid which has been watched.

In [ ]:
from constants import data_path, entities_path, relations_path, Path
import pandas as pd

In [ ]:
sample_data = Path("mooc_cube_x-100k")

In [ ]:
user2video = pd.read_parquet(sample_data / "user-video.parquet.gzip")
# Actual dataset
# user2video = pd.read_json(data_path / "user-video.json", lines=True)

In [ ]:
video_id2ccid = pd.read_csv(relations_path / "video_id-ccid.txt", sep="\t", names=["video_id", "ccid"])

In [ ]:
usr_vid_exploded = user2video.explode("seq")
user_video_ccid = usr_vid_exploded.assign(video_id=usr_vid_exploded["seq"].str["video_id"]).merge(video_id2ccid).sort_values("user_id")

Generating the table showing how many times a user has re-watched a given video, 
potentially across courses.

In [ ]:
user_video_count = user_video_ccid.value_counts(["ccid", "user_id"]).reset_index().rename(columns={0: "num_watched"})
user_video_count

### Baseline: Implicit feedback
Testing out baseline models for repetition, only considering repeated/not


In [ ]:
user_video_reconsumed = (user_video_count[user_video_count["num_watched"] > 1]
                    .rename(columns={"user_id": "user", "ccid": "item", "num_watched": "rating"}))#.pivot(index="user_id", columns="ccid", values="num_watched")
user_video_reconsumed#[user_video_reconsumed.index =="U_10232296"].dropna(axis=1)#.iloc[0, :]#.notna().count()

In [ ]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, item_knn, user_knn, als

In [ ]:
algo_ii = item_knn.ItemItem(10)
algo_uu = user_knn.UserUser(10)
algo_als = als.ImplicitMF(50)
# algo_als = 

In [ ]:
def eval(aname, algo, train, test):
    """Generates recommendations over one partition of the dataset"""
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 10)
    # add the algorithm name for analyzability
    recs['algorithm'] = aname
    return recs

In [ ]:
all_recs = []
test_data = []
partitioned = xf.partition_users(user_video_reconsumed, 5, xf.SampleN(1))
for idx, (train, test) in enumerate(partitioned):
    test_data.append(test)
    # all_recs.append(eval('ItemItem', algo_ii, train, test))
    # all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs.append(eval('ImplicitALS', algo_als, train, test))

In [ ]:
test_results = pd.concat(test_data, ignore_index=True)

In [ ]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
results = rla.compute(all_recs, test_data)
results.head()

In [ ]:
results.groupby('algorithm').ndcg.mean()